In [16]:
# public packages
import andes
import ams
import cvxpy as cp
import numpy as np
import gurobipy

# local packages
import anci_fun as af

# v1 try to include dynamic constraints using reduced input/output NN model

# ------ nn information ------
# input:   [M, D]
# output:  [rocof_max, fnadir]
# nn_diam: [16, 128, 2]

# Para preparetion

In [17]:
# neural network parameters 
nn_name = 'trained_model.pth'
nn_weights, nn_bias, nn_diam = af.extract_nn_para(nn_name) # three list

In [18]:
# MD parameters
vsg_num = 8

# lb_M = np.array([0, 0, 0, 0, 0, 0, 0, 0])
lb_M = np.array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])
ub_M = np.array([6, 6, 6, 6, 6, 6, 6, 6])

# lb_D = np.array([0, 0, 0, 0, 0, 0, 0, 0])
lb_D = np.array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])*5
ub_D = np.array([4, 4, 4, 4, 4, 4, 4, 4])

In [19]:
sf_load = 1.0

# y = [rocof_max, fnadir, dtheta_max, eig_max]
lb_y = np.array([-0.5, -0.6])
ub_y = np.array([ 0.5,  0.6])

# Load ams model

In [20]:
ssp = ams.load('ams_case123_REGCV1_v5.xlsx')

ssp.LDOPF.run()

ssp.LDOPF.om.mdl.solve()

ssp.LDOPF.om.vars

LDOPF solved as optimal in 0.0890 seconds with exit code 0.


OrderedDict([('pg', Variable((11,))),
             ('pn', Variable((117,))),
             ('qg', Variable((11,))),
             ('qn', Variable((117,))),
             ('vsq', Variable((117,))),
             ('pl', Variable((116,))),
             ('ql', Variable((116,)))])

# Define new variables, constraints, and obj

In [21]:
# M and D

M = cp.Variable(vsg_num, name="M")
D = cp.Variable(vsg_num, name="D")

constraints_M = [M >= lb_M, M <= ub_M]
constraints_D = [D >= lb_D, D <= ub_D]

In [22]:
# data for the neural network linearization
pg    = ssp.LDOPF.om.pg

x1    = cp.hstack([M, D])  # this version doesn't incelu load scaling factor

w1    = nn_weights[0]
w2    = nn_weights[1]

b1    = nn_bias[0]
b2    = nn_bias[1]

hdown = -100
hup   = 100

In [23]:
# neural network linization

z = cp.Variable(nn_diam[1], name="z")
a = cp.Variable(nn_diam[1], name="a", boolean=True)

z_ = w1 @ x1 + b1

y = w2 @ z + b2

constraints_nn = [
                    # linization constraints
                    z <= z_ - hdown * (1 - a),
                    z >= z_,
                    z <= hup * a,
                    z >= 0,
                    # output constraints
                    # y[0] <= ub_y[0],       # RoCoF
                    y[0] >= lb_y[0],
                    y[1] <= ub_y[1],       # fnadir
                    y[1] >= lb_y[1],
                ]

In [24]:
# extract original ams constraints
constriants_ams = [ssp.LDOPF.om.constrs[cname] for cname in ssp.LDOPF.om.constrs.keys()]

In [25]:
constraints = constraints_M + constraints_D + constraints_nn + constriants_ams

# constraints = constriants_ams + constraints_M + constraints_D

# Formulate new problem

pg = ['PV_1',
 'PV_10',
 'PV_11',
 'PV_2',
 'PV_3',
 'PV_4',
 'PV_5',
 'PV_6',
 'PV_7',
 'PV_8',
 'PV_9']

 SG: PV_1, PV_2, PV_3 (slack bus)

 IBR: PV_4, PV_5, ..., PV_11

In [26]:
c2 = np.array([0.01, 0, 0, 0.01, 0.02, 0, 0, 0, 0, 0, 0])*100
c1 = np.array([  20, 0, 0,   20,   30, 0, 0, 0, 0, 0, 0])*10

# d  = np.array([0, 0, 0, 0.01, 0, 0, 0.001, 0])
c_M  = np.array([0.02, 0.01, 0.05, 0.01, 0.06, 0.07, 0.01, 0.02])

c_D  = np.array([0.01, 0.015, 0.01, 0.01, 0.01, 0.01, 0.02, 0.01])

pg_power = pg**2

#cost = cp.sum(c2 @ pg_power + c1 @ pg)

cost = cp.sum(c2 @ pg_power + c1 @ pg + c_M @ M + c_D @ D)

obj = cp.Minimize(cost)

In [27]:
# Create the optimization problem
prob = cp.Problem(obj, constraints)

# Solve the optimization problem

# prob.solve(solver=cp.GUROBI, verbose=True)

prob.solve(solver=cp.ECOS_BB, verbose=True)

# prob.solve()

                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Sep 11 10:54:09 PM: Your problem has 877 variables, 24 constraints, and 0 parameters.
(CVXPY) Sep 11 10:54:09 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 11 10:54:09 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 11 10:54:09 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 11 10:54:09 PM: Compiling problem (target solver=ECOS_BB).
(CVXPY) Sep 11 10:54:09 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffi

 8  +2.378e+01  -4.975e+02  +3e+02  7e-06  8e-06  1e+00  3e-01  0.8340  1e-01   1  1  2 |  0  0
 9  +2.881e+01  -1.987e+02  +6e+01  5e-06  3e-06  2e+00  5e-02  0.9159  4e-02   1  2  1 |  0  0
10  +1.281e+02  +7.059e+00  +3e+00  5e-06  2e-06  4e+01  2e-03  0.9610  1e-03   1  1  2 |  0  0
11  +3.096e+03  +5.108e+03  +6e-02  1e-05  2e-06  2e+03  4e-05  0.9843  3e-03   2  1  1 |  0  0
12  +2.741e+05  +4.703e+05  +6e-04  3e-05  2e-06  2e+05  5e-07  0.9890  1e-04   1  2  2 |  0  0
13  +2.476e+07  +4.250e+07  +7e-06  8e-04  2e-06  2e+07  5e-09  0.9890  1e-04   1  3  2 |  0  0

PRIMAL INFEASIBLE (within feastol=1.2e-10).
Runtime: 0.088014 seconds.


ECOS 2.0.10 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +4.392e+02  -2.896e+06  +3e+06  3e-02  5e-01  1e+00  2e+03    ---    ---    1  1  - |  -  - 
 1  -1.802e+01  -1.096e+06  +1e+06  1e-02  2e-01  2e+02  9e+02  0.71

In [ ]:
ssp.LDOPF.om.pg.value

array([0.02    , 0.034125, 0.034125, 0.02    , 0.04    , 0.034125,
       0.034125, 0.034125, 0.034125, 0.034125, 0.034125])

In [ ]:
M.value

array([0.0550819 , 0.05      , 0.05      , 0.18360076, 0.07189779,
       0.05      , 0.23838847, 0.2688555 ])

In [ ]:
D.value

array([0.08507053, 0.16234449, 0.12785225, 0.14253555, 0.29025298,
       0.05      , 0.05      , 0.075514  ])

# 